In [ ]:
!python --version

In [ ]:
import re
import glob
import json
import unicodedata
import pandas as pd

In [ ]:
# txt -> csv
file_list = glob.glob("*.txt")
row_data = []

for file in file_list:
  file_name = "text_list_csv/" + file[:13] + ".csv"
  with open(file)as f:
    text = f.read()
    text = text.replace('\n', '').replace(' ', '')
    text = re.sub('【\d{4}】', '', text)
  # text = "(１２３４)から（１）、（２）を抽出"

  c1 = ""
  c2 = ""
  text = unicodedata.normalize('NFKC', text).replace('\u309a', '').replace('\u2014', '')
  if re.search('(?<=【発明を実施するための形態】)(.*?)(?=【\w+】)', text):
    c1 = re.search('(?<=【発明を実施するための形態】)(.*?)(?=【\w+】)', text).group()
  if re.search('(?<=【実施例】)(.*?)(?=【\w+】)', text):
    c2 = re.search('(?<=【実施例】)(.*?)(?=【\w+】)', text).group()
  text = c1 + c2
  b_text =  [x + '\n\n' for x in text[0::1200]]
  with open('sample999.txt', 'w', encoding='shift-jis')as f:
    f.write(text)
  sentence_list = []
  for s in text.split('。'):
    if len(s) > 256:
      for pos in range(0, len(s), 256):
        sentence_list.append(s[pos:pos + 256])
    else:
      sentence_list.append(s)
  dic_for_df = {}
  dic_for_df['text'] = sentence_list
  df = pd.DataFrame(dic_for_df)
  df.to_csv(file_name, index=False, encoding='shift-jis')

In [ ]:
# csv結合
file_lst = glob.glob('path/to/*.csv')
data_lst = []
for file in file_lst:
  data_lst.append(pd.read_csv(file, encoding='shift-jis', header=None).drop(0))
df = pd.concat(data_lst, axis=0, sort=True)
csv_name = 'hsec_total_uq.csv'
df.to_csv(csv_name, encoding='shift-jis', index=None, header=None)

In [ ]:
file = pd.read_csv(csv_name, encoding='shift-jis', header=None)
dict_lst = []
for col in file.values:
  d = {}
  text = col[0]
  d['text'] = text
  wordlist = []
  for w in col[1:]:
    if not pd.isna(w):
      wordlist.append(w)

  elist = []
  for e in wordlist:
    worddict = {}
    worddict['name'] = e
    end_pos = text.find(e) + len(e)
    worddict['span'] = [text.find(e), end_pos]
    id = "技術語"
    worddict['type'] = id
    elist.append(worddict)
  d['entities'] = elist
  dict_lst.append(d)
print(dict_lst)

with open('json_name.json', 'w') as f:
  json.dump(dict_lst, f, indent=2, ensure_ascii=False)

In [ ]:
# 文字位置の観測
# 部分一致語、同一単語による位置の重複を避けるため、文字列の検索位置を適宜更新する。
# プログラムの仕様上、位置情報を知りたい語句がリスト内に出現順に並んでいる必要がある。
dataset = []
uq_words = ['偏光子保護フィルム', 'ポリエステルフィルム', 'フィルム']  #　テキスト内の固有表現

sample = {}
text = "本発明の偏光子保護フィルムは、ポリエステルフィルムであり、フィルムの流れ方向又は幅方向に対する熱収縮率が最大となる方向の傾きの絶対値が15度以下であることが好ましい" #サンプルのテキスト
sample['text'] = text

e_item = [] #entitiesに収容される要素のひとまとまり
next_search_spos = 0
for entity in uq_words:
  e_info = {}
  e_info['name'] = entity
  end_pos = len(text[:next_search_spos]) + text[next_search_spos:].find(entity) + len(entity)
  e_info['span'] = [len(text[:next_search_spos]) + text[next_search_spos:].find(entity), end_pos]
  id = '技術語'
  e_info['type'] = id
  e_item.append(e_info)
  next_search_spos = end_pos
sample['entities'] = e_item
dataset.append(sample)

print(text[4:13])
print(dataset)

In [ ]:
word_list = ["カモシカ", "シカ", "アシカ"]
text = "カモシカもシカも確かに鹿だが、アシカは確か鹿ではない"

#変数を用意して繰り返しの都度、検索する範囲を変更する
next_search_spos = 0
for word in word_list:
  start_pos = len(text[:next_search_spos]) + text[next_search_spos:].find(word) # 開始位置
  end_pos =  len(text[:next_search_spos]) + text[next_search_spos:].find(word) + len(word) # 終了位置
  print(word, start_pos, end_pos) # (リスト要素, 開始位置, 終了位置)
  next_search_spos = end_pos